First we need to have a look at the data we have to model the problem. The pandas function pandas.

In [1]:
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import numpy as np


In [2]:
%bash

gsutil cp gs://advanced-solutions-lab/Electricity/RealMarketPriceData.csv .
gsutil cp gs://advanced-solutions-lab/Electricity/WeatherHistoricalData.zip .
unzip WeatherHistoricalData.zip
rm WeatherHistoricalData.zip

Archive:  WeatherHistoricalData.zip


Copying gs://advanced-solutions-lab/Electricity/RealMarketPriceData.csv...
/ [1 files][218.0 KiB/218.0 KiB]                                                
Operation completed over 1 objects/218.0 KiB.                                    
Copying gs://advanced-solutions-lab/Electricity/WeatherHistoricalData.zip...
/ [1 files][  4.7 MiB/  4.7 MiB]                                                
Operation completed over 1 objects/4.7 MiB.                                      
replace WeatherHistoricalData/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: new name: replace rHistoricalData.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [3]:
price = pd.read_csv('RealMarketPriceData.csv', sep=";")
weather = pd.read_csv('./WeatherHistoricalData/historical_weather.csv')
points = pd.read_csv('./WeatherHistoricalData/points.csv')

In [4]:
pdp.ProfileReport(price)

/usr/local/envs/py2env/lib/python2.7/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
/usr/local/envs/py2env/lib/python2.7/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


Number of variables,2
Number of observations,9552
Total Missing (%),0.0%
Total size in memory,149.3 KiB
Average record size in memory,16.0 B
Numeric,1
Categorical,0
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [5]:
pdp.ProfileReport(weather)


Number of variables,12
Number of observations,272592
Total Missing (%),0.0%
Total size in memory,25.0 MiB
Average record size in memory,96.0 B
Numeric,8
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,3


In [6]:
price.columns = ['date', 'price']
price.head()


,date,price
0,01/01/2015 00:00,48.10
1,01/01/2015 01:00,47.33
2,01/01/2015 02:00,42.27
3,01/01/2015 03:00,38.41
4,01/01/2015 04:00,35.72


In [7]:
from datetime import datetime
price['date'] = price.apply(lambda x : datetime.strptime(x['date'], '%d/%m/%Y %H:%M'), axis=1)

In [8]:
price.head()

,date,price
0,2015-01-01 00:00:00,48.10
1,2015-01-01 01:00:00,47.33
2,2015-01-01 02:00:00,42.27
3,2015-01-01 03:00:00,38.41
4,2015-01-01 04:00:00,35.72


In [9]:
RANGE_LENGTH = 3
min_date = price['date'].min()

price['range_id'] = price.apply(lambda x : (x['date']-min_date).days/RANGE_LENGTH, axis=1)
                                            

In [20]:

np.random.seed(1)
test_ids = np.random.choice(price['range_id'].unique(), size=int(0.1*price.range_id.max()), replace=False)
price['split'] = price.range_id.apply(lambda x: 1 if x in test_ids else 0)
mask = price.split==0
eval_ids = np.random.choice(price[mask].range_id.unique(), size=int(0.1*price.range_id.max()), replace=False)
price.split = price.apply(lambda row: 2 if row["range_id"] in eval_ids else row["split"], axis=1)


To https://source.developers.google.com/p/qwiklabs-gcp-0708abec6d37116e/r/POL_GER_Energy_Repo
 * [new branch]      master -> master
